# AutoGluon: Semantic Segmentation

## Objective
This notebook demonstrates **semantic segmentation** using AutoGluon. Semantic segmentation classifies each pixel in an image into a category, creating a pixel-wise segmentation map.

## Use Case
Semantic segmentation is useful for:
- Autonomous driving (road, sidewalk, vehicle, pedestrian segmentation)
- Medical imaging (organ segmentation, tumor boundary detection)
- Satellite imagery analysis (land use classification, building detection)
- Agriculture (crop field segmentation, plant disease detection)
- Video editing (background removal, scene understanding)

## Key Concepts
- Pixel-level classification
- Each pixel assigned to a semantic class
- Output is a segmentation mask with same dimensions as input
- Common architectures: U-Net, DeepLab, FCN

In [ ]:
# Install AutoGluon
!pip install -q autogluon

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
from autogluon.tabular import TabularDataset, TabularPredictor
import os
import shutil

In [ ]:
# Load dataset
# TODO: Upload your semantic segmentation dataset or use URL
# Dataset should have:
# - Images (input photos)
# - Segmentation masks (ground truth labels per pixel)
#
# Format options:
# 1. CSV with columns: 'image' (path to input), 'mask' (path to segmentation mask)
# 2. Directory structure: /images/ and /masks/ folders with matching filenames

# Example: train_data = TabularDataset('path/to/segmentation_data.csv')

train_data = None  # Replace with your data
test_data = None   # Replace with your data

print("Dataset loaded successfully!")
if train_data is not None:
    print(f"Training data shape: {train_data.shape}")
    print(train_data.head())

In [ ]:
# Set label column
# For semantic segmentation, this typically points to the mask/label image
LABEL = 'mask'  # TODO: Replace with your mask column name

In [ ]:
# Auto-detect problem type based on label
# Semantic segmentation is a specialized classification task (per-pixel)
if train_data is not None and LABEL in train_data.columns:
    # Check if the label is numeric (regression) or categorical (classification)
    if pd.api.types.is_numeric_dtype(train_data[LABEL]):
        # Check if it's continuous or discrete
        unique_ratio = train_data[LABEL].nunique() / len(train_data)
        if unique_ratio > 0.05:  # More than 5% unique values suggests regression
            problem_type = 'regression'
            eval_metric = 'rmse'
        else:
            problem_type = 'classification'
            eval_metric = 'roc_auc'
    else:
        problem_type = 'classification'
        eval_metric = 'roc_auc'
else:
    # Default for semantic segmentation
    problem_type = 'classification'
    eval_metric = 'roc_auc'  # Can also use IoU (Intersection over Union)

print(f"Problem Type: {problem_type} (Semantic Segmentation)")
print(f"Evaluation Metric: {eval_metric}")
print("\nNote: Semantic segmentation classifies each pixel in the image.")

In [ ]:
# Train the model
# AutoGluon will use semantic segmentation models like U-Net, DeepLab, etc.
predictor = TabularPredictor(
    label=LABEL,
    problem_type=problem_type,
    eval_metric=eval_metric,
    path='./autogluon-segmentation-model'
).fit(
    train_data=train_data,
    presets='medium_quality',
    time_limit=900
)

print("Model training completed!")

In [ ]:
# Display and save leaderboard
leaderboard = predictor.leaderboard(test_data, silent=True)
print("\nModel Leaderboard:")
print(leaderboard)

# Save leaderboard to CSV
leaderboard.to_csv('leaderboard.csv', index=False)
print("\nLeaderboard saved to leaderboard.csv")

In [ ]:
# Display and save feature importance
try:
    feature_importance = predictor.feature_importance(test_data)
    print("\nFeature Importance:")
    print(feature_importance)
    
    # Save feature importance to CSV
    feature_importance.to_csv('feature_importance.csv')
    print("\nFeature importance saved to feature_importance.csv")
except Exception as e:
    print(f"Could not compute feature importance: {e}")

In [ ]:
# Make predictions
if test_data is not None:
    predictions = predictor.predict(test_data)
    print("\nPredictions (Segmentation Masks):")
    print(predictions.head())
    
    # Semantic segmentation output is typically:
    # - A mask image with same dimensions as input
    # - Each pixel value represents a class label
    
    print("\nSegmentation Output Format:")
    print("Output is a mask where each pixel value represents a semantic class.")
    print("Example classes: 0=background, 1=person, 2=car, 3=road, etc.")
    
    # Example: Visualize segmentation
    print("\nTo visualize segmentation results:")
    print("# Load mask image and overlay on original image")
    print("# Use color mapping for different classes")

In [ ]:
# Save model artifacts as zip file
model_path = './autogluon-segmentation-model'
zip_filename = 'autogluon_segmentation_model'

if os.path.exists(model_path):
    shutil.make_archive(zip_filename, 'zip', model_path)
    print(f"\nModel artifacts saved to {zip_filename}.zip")
else:
    print("Model path not found. Train the model first.")